### import

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist
from time import time
import os
print(tf.__version__)

2.3.0


### Checkpoint function

In [2]:
def load(model, checkpoint_dir):
    print(" [*] Reading checkpoints...")
    
    ckpt = tf.train.get_checkpoint_state(checkpoint_dir) # 해당 dir에 학습해둔 모델 있는지 확인
    if ckpt:
        ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
        checkpoint = tf.train.Checkpoint(dnn=model)
        checkpoint.restore(save_path=os.path.join(checkpoint_dir, ckpt_name))
        counter = int(ckpt_name.split('-')[1])
        print(" [*] Success to read {}".format(ckpt_name))
        return True, counter
    else:
        print(" [*] Failed to find a checkpoint")
        return False, 0
    
def check_folder(dir):
    if not os.path.exists(dir):
        os.makedirs(dir)
    return dir

### Data load & pre-processing function

In [3]:
def load_mnist():
    (train_data, train_labels), (test_data, test_labels) = mnist.load_data()
    train_data = np.expand_dims(train_data, axis=-1) # [N,28,28] -> [N,28,28,1]
    test_data = np.expand_dims(test_data, axis=-1) # [N,28,28] -> [N,28,28,1]
    
    train_data, test_data = normalize(train_data, test_data)
    
    train_labels = to_categorical(train_labels, 10) # [N,] -> [N, 10]
    test_labels = to_categorical(test_labels, 10) # [N,] -> [N, 10]
    
    return train_data, train_labels, test_data, test_labels

def normalize(train_data, test_data):
    train_data = train_data.astype(np.float32) / 255.0
    test_data = test_data.astype(np.float32) / 255.0
    return train_data, test_data

### Performance function

In [11]:
def loss_func(model, images, labels):
    logits = model(images, training=True)
    loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y_pred=logits,
                                                                   y_true=labels,
                                                                   from_logits=True)) # softmax를 거치지 않을때
    return loss

def accuracy_func(model, images, labels):
    logits = model(images, training=False)
    prediction = tf.equal(tf.argmax(logits, -1), tf.argmax(labels, -1))
    accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32))
    return accuracy

def grad(model, images, labels):
    with tf.GradientTape() as tape:
        loss = loss_func(model, images, labels)
    return tape.gradient(loss, model.trainable_variables)

### Model function

In [18]:
def flatten():
    return tf.keras.layers.Flatten()

def dense(label_dim, weight_init):
    return tf.keras.layers.Dense(units=label_dim, # 출력값의 크기
                                 use_bias=True, # 편향 사용 유무
                                 kernel_initializer=weight_init) # 가중치 초기화 함수

def relu():
    return tf.keras.layers.Activation(tf.keras.activations.relu)

def batch_norm():
    return tf.keras.layers.BatchNormalization()

### Create model(class version)

In [20]:
class create_model_class(tf.keras.Model):
    def __init__(self, label_dim):
        super(create_model_class, self).__init__()
        weight_init = tf.keras.initializers.glorot_uniform() 
        # Sigmoid : Xavier 초기화
        # Relu : He 초기화
        
        self.model = tf.keras.Sequential()
        self.model.add(flatten())
        
        for i in range(4):
            self.model.add(dense(512, weight_init))
            self.model.add(batch_norm())
            self.model.add(relu())
            
        self.model.add(dense(label_dim, weight_init))
        
    def call(self, x, training=None, mask=None):
        x = self.model(x)
        return x

### Create model(function version)

In [21]:
def create_model_function(label_dim):
    weight_init = tf.keras.initializers.glorot_uniform()
    
    model = tf.keras.Sequential()
    model.add(flatten())
    
    for i in range(4):
        model.add(dense(512, weight_init))
        model.add(batch_norm())
        model.add(relu())
        
    model.add(dense(label_dim, weight_init))
    
    return model

### Define data & hyper-parameter

In [22]:
# dataset
train_x, train_y, test_x, test_y = load_mnist()

# parameters
learning_rate = 0.001
batch_size = 128

training_epochs = 1
training_iterations = len(train_x) // batch_size

label_dim = 10
train_flag = True

# Graph input using Dataset API
# prefetch : data 로드 시간을 줄이기 위해 미리 batch size 만큼 적재시킴
# drop_remainder : 마지막 남은 data를 drop 여부
train_dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y)).\
                                                   shuffle(buffer_size=100000).\
                                                   prefetch(buffer_size=batch_size).\
                                                   batch(batch_size, drop_remainder=True) 


test_dataset = tf.data.Dataset.from_tensor_slices((test_x, test_y)).\
                                                  shuffle(buffer_size=100000).\
                                                  prefetch(buffer_size=len(test_x)).\
                                                  batch(len(test_x))

### Define model & optimizer & writer

In [23]:
# Model
network = create_model_function(label_dim)

# Training
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

# Writer
checkpoint_dir = 'checkpoints'
logs_dir = 'logs'

model_dir = 'nn_batchnorm'

checkpoint_dir = os.path.join(checkpoint_dir, model_dir)
check_folder(checkpoint_dir)
checkpoint_prefix = os.path.join(checkpoint_dir, model_dir)
logs_dir = os.path.join(logs_dir, model_dir)

### Restore checkpoint & start train or test phase

In [25]:
if train_flag:
    checkpoint = tf.train.Checkpoint(dnn=network)
    
    # create writer for tensorboard
    summary_writer = tf.summary.create_file_writer(logdir=logs_dir)
    start_time = time()
    
    # restore check-point if it exits
    could_load, checkpoint_counter = load(network, checkpoint_dir)
    
    if could_load:
        start_epoch = (int)(checkpoint_counter / training_iterations)
        counter = checkpoint_counter
        print(" [*] Load SUCCESS")
    else:
        start_epoch = 0
        start_iteration = 0
        counter = 0
        print(" [!] Load failed...")
        
    # train phase
    with summary_writer.as_default():
        for epoch in range(start_epoch, training_epochs):
            for idx, (train_input, train_label) in enumerate(train_dataset):
                grads = grad(network, train_input, train_label)
                optimizer.apply_gradients(grads_and_vars=zip(grads, network.trainable_variables))
                
                train_loss = loss_func(network, train_input, train_label)
                train_accuracy = accuracy_func(network, train_input, train_label)
                
                for test_input, test_label in test_dataset:
                    test_accuracy = accuracy_func(network, test_input, test_label)
                    
                tf.summary.scalar(name='train_lass', data=train_loss, step=counter)
                tf.summary.scalar(name='train_accuracy', data=train_accuracy, step=counter)
                tf.summary.scalar(name='test_accuracy', data=test_accuracy, step=counter)
                
                print(
                    "Epoch: [%2d] [%5d/%5d] time: %4.4f, train_loss: %.8f, train_accuracy: %.4f, test_Accuracy: %.4f" \
                    % (epoch, idx, training_iterations, time() - start_time, train_loss, train_accuracy, test_accuracy))
                
                counter += 1
        checkpoint.save(file_prefix=checkpoint_prefix + '-{}'.format(counter))   
# test phase
else:
    _, _ = load(network, checkpoint_dir)
    for test_input, test_label in test_dataset:
        test_accuracy = accuracy_func(network, test_input, test_label)
        
    print("test Accuracy: %.4f" % (test_accuracy))

 [*] Reading checkpoints...
 [*] Failed to find a checkpoint
 [!] Load failed...
Epoch: [ 0] [    0/  468] time: 0.6497, train_loss: 0.31145769, train_accuracy: 0.4844, test_Accuracy: 0.3284
Epoch: [ 0] [    1/  468] time: 0.9689, train_loss: 0.29828429, train_accuracy: 0.4531, test_Accuracy: 0.4346
Epoch: [ 0] [    2/  468] time: 1.3534, train_loss: 0.39583522, train_accuracy: 0.4844, test_Accuracy: 0.4965
Epoch: [ 0] [    3/  468] time: 1.6960, train_loss: 0.28918177, train_accuracy: 0.6641, test_Accuracy: 0.5557
Epoch: [ 0] [    4/  468] time: 2.0444, train_loss: 0.28164589, train_accuracy: 0.6094, test_Accuracy: 0.6357
Epoch: [ 0] [    5/  468] time: 2.3704, train_loss: 0.24202128, train_accuracy: 0.6875, test_Accuracy: 0.7040
Epoch: [ 0] [    6/  468] time: 2.7042, train_loss: 0.41021007, train_accuracy: 0.6719, test_Accuracy: 0.7510
Epoch: [ 0] [    7/  468] time: 3.0439, train_loss: 0.18391332, train_accuracy: 0.8359, test_Accuracy: 0.7756
Epoch: [ 0] [    8/  468] time: 3.3523,

Epoch: [ 0] [   74/  468] time: 28.6883, train_loss: 0.08724945, train_accuracy: 0.9609, test_Accuracy: 0.9347
Epoch: [ 0] [   75/  468] time: 29.0787, train_loss: 0.14508435, train_accuracy: 0.9141, test_Accuracy: 0.9333
Epoch: [ 0] [   76/  468] time: 29.5442, train_loss: 0.12469251, train_accuracy: 0.9531, test_Accuracy: 0.9339
Epoch: [ 0] [   77/  468] time: 29.9727, train_loss: 0.18417816, train_accuracy: 0.9219, test_Accuracy: 0.9350
Epoch: [ 0] [   78/  468] time: 30.4275, train_loss: 0.14218293, train_accuracy: 0.9688, test_Accuracy: 0.9350
Epoch: [ 0] [   79/  468] time: 30.9048, train_loss: 0.09429166, train_accuracy: 0.9531, test_Accuracy: 0.9365
Epoch: [ 0] [   80/  468] time: 31.3898, train_loss: 0.11751028, train_accuracy: 0.9453, test_Accuracy: 0.9375
Epoch: [ 0] [   81/  468] time: 31.8251, train_loss: 0.08378951, train_accuracy: 0.9766, test_Accuracy: 0.9395
Epoch: [ 0] [   82/  468] time: 32.2712, train_loss: 0.13207969, train_accuracy: 0.9609, test_Accuracy: 0.9410
E

Epoch: [ 0] [  148/  468] time: 58.5988, train_loss: 0.12361617, train_accuracy: 0.9531, test_Accuracy: 0.9503
Epoch: [ 0] [  149/  468] time: 58.9667, train_loss: 0.12273595, train_accuracy: 0.9531, test_Accuracy: 0.9510
Epoch: [ 0] [  150/  468] time: 59.3454, train_loss: 0.10580310, train_accuracy: 0.9688, test_Accuracy: 0.9506
Epoch: [ 0] [  151/  468] time: 59.7485, train_loss: 0.09127536, train_accuracy: 0.9531, test_Accuracy: 0.9503
Epoch: [ 0] [  152/  468] time: 60.1467, train_loss: 0.09576407, train_accuracy: 0.9609, test_Accuracy: 0.9504
Epoch: [ 0] [  153/  468] time: 60.5430, train_loss: 0.08847961, train_accuracy: 0.9766, test_Accuracy: 0.9500
Epoch: [ 0] [  154/  468] time: 60.9138, train_loss: 0.11034408, train_accuracy: 0.9688, test_Accuracy: 0.9497
Epoch: [ 0] [  155/  468] time: 61.3033, train_loss: 0.08666529, train_accuracy: 0.9609, test_Accuracy: 0.9505
Epoch: [ 0] [  156/  468] time: 61.6829, train_loss: 0.18331331, train_accuracy: 0.9609, test_Accuracy: 0.9522
E

Epoch: [ 0] [  222/  468] time: 90.5394, train_loss: 0.08651230, train_accuracy: 0.9453, test_Accuracy: 0.9551
Epoch: [ 0] [  223/  468] time: 90.9239, train_loss: 0.12799431, train_accuracy: 0.9297, test_Accuracy: 0.9577
Epoch: [ 0] [  224/  468] time: 91.3465, train_loss: 0.06088179, train_accuracy: 0.9844, test_Accuracy: 0.9590
Epoch: [ 0] [  225/  468] time: 91.7652, train_loss: 0.04807514, train_accuracy: 0.9766, test_Accuracy: 0.9604
Epoch: [ 0] [  226/  468] time: 92.2259, train_loss: 0.13994578, train_accuracy: 0.9688, test_Accuracy: 0.9602
Epoch: [ 0] [  227/  468] time: 92.6387, train_loss: 0.03939215, train_accuracy: 0.9688, test_Accuracy: 0.9602
Epoch: [ 0] [  228/  468] time: 93.0574, train_loss: 0.10947061, train_accuracy: 0.9844, test_Accuracy: 0.9585
Epoch: [ 0] [  229/  468] time: 93.4918, train_loss: 0.15010038, train_accuracy: 0.9609, test_Accuracy: 0.9585
Epoch: [ 0] [  230/  468] time: 93.9778, train_loss: 0.11719492, train_accuracy: 0.9688, test_Accuracy: 0.9592
E

Epoch: [ 0] [  296/  468] time: 120.5426, train_loss: 0.13247252, train_accuracy: 0.9531, test_Accuracy: 0.9614
Epoch: [ 0] [  297/  468] time: 120.9057, train_loss: 0.13777652, train_accuracy: 0.9844, test_Accuracy: 0.9631
Epoch: [ 0] [  298/  468] time: 121.2746, train_loss: 0.05565608, train_accuracy: 0.9844, test_Accuracy: 0.9646
Epoch: [ 0] [  299/  468] time: 121.6630, train_loss: 0.10116437, train_accuracy: 0.9844, test_Accuracy: 0.9662
Epoch: [ 0] [  300/  468] time: 122.0622, train_loss: 0.05710784, train_accuracy: 0.9766, test_Accuracy: 0.9662
Epoch: [ 0] [  301/  468] time: 122.4760, train_loss: 0.08938822, train_accuracy: 0.9766, test_Accuracy: 0.9660
Epoch: [ 0] [  302/  468] time: 122.8772, train_loss: 0.16464682, train_accuracy: 0.9531, test_Accuracy: 0.9655
Epoch: [ 0] [  303/  468] time: 123.2442, train_loss: 0.12145168, train_accuracy: 0.9609, test_Accuracy: 0.9648
Epoch: [ 0] [  304/  468] time: 123.6199, train_loss: 0.05968095, train_accuracy: 0.9844, test_Accuracy:

Epoch: [ 0] [  370/  468] time: 149.3941, train_loss: 0.08555359, train_accuracy: 0.9766, test_Accuracy: 0.9620
Epoch: [ 0] [  371/  468] time: 149.7972, train_loss: 0.05151026, train_accuracy: 0.9766, test_Accuracy: 0.9617
Epoch: [ 0] [  372/  468] time: 150.2023, train_loss: 0.08251015, train_accuracy: 0.9688, test_Accuracy: 0.9612
Epoch: [ 0] [  373/  468] time: 150.6005, train_loss: 0.05639978, train_accuracy: 0.9766, test_Accuracy: 0.9604
Epoch: [ 0] [  374/  468] time: 150.9713, train_loss: 0.07504183, train_accuracy: 0.9688, test_Accuracy: 0.9592
Epoch: [ 0] [  375/  468] time: 151.3344, train_loss: 0.12118610, train_accuracy: 0.9609, test_Accuracy: 0.9594
Epoch: [ 0] [  376/  468] time: 151.7073, train_loss: 0.11052380, train_accuracy: 0.9688, test_Accuracy: 0.9603
Epoch: [ 0] [  377/  468] time: 152.0830, train_loss: 0.07065046, train_accuracy: 0.9531, test_Accuracy: 0.9616
Epoch: [ 0] [  378/  468] time: 152.4705, train_loss: 0.07930412, train_accuracy: 0.9453, test_Accuracy:

Epoch: [ 0] [  444/  468] time: 178.3101, train_loss: 0.06250580, train_accuracy: 0.9766, test_Accuracy: 0.9700
Epoch: [ 0] [  445/  468] time: 178.6858, train_loss: 0.09479441, train_accuracy: 0.9609, test_Accuracy: 0.9696
Epoch: [ 0] [  446/  468] time: 179.0802, train_loss: 0.04028416, train_accuracy: 0.9844, test_Accuracy: 0.9685
Epoch: [ 0] [  447/  468] time: 179.4754, train_loss: 0.03405234, train_accuracy: 0.9844, test_Accuracy: 0.9673
Epoch: [ 0] [  448/  468] time: 179.8902, train_loss: 0.10113324, train_accuracy: 0.9609, test_Accuracy: 0.9663
Epoch: [ 0] [  449/  468] time: 180.2923, train_loss: 0.04755982, train_accuracy: 0.9844, test_Accuracy: 0.9660
Epoch: [ 0] [  450/  468] time: 180.6886, train_loss: 0.05363964, train_accuracy: 0.9766, test_Accuracy: 0.9651
Epoch: [ 0] [  451/  468] time: 181.0790, train_loss: 0.08492129, train_accuracy: 0.9688, test_Accuracy: 0.9646
Epoch: [ 0] [  452/  468] time: 181.4577, train_loss: 0.10405318, train_accuracy: 0.9688, test_Accuracy:

### Test accuracy : 95.73%